In [1]:
from typing_extensions import Coroutine
import requests
from bs4 import BeautifulSoup
import pandas as pd
import functools as ft
import numpy as np

if __name__ == '__main__':

  diariosRequest = requests.get('https://datos.gob.ar/dataset/cultura-mapa-cultural-agentes-actividades-culturales/archivo/cultura_0d1b731b-28c9-4402-a7eb-018c9cf8958b')
  cineRequest = requests.get('https://datos.gob.ar/dataset/cultura-mapa-cultural-espacios-culturales/archivo/cultura_f7a8edb8-9208-41b0-8f19-d72811dcea97')
  editorialesRequest = requests.get('https://datos.gob.ar/dataset/cultura-mapa-cultural-agentes-actividades-culturales/archivo/cultura_2bcb7ee9-1c75-4a37-adec-2e8c822d2e3d')

  diarios_sopa = BeautifulSoup(diariosRequest.content, 'html.parser')
  cine_sopa = BeautifulSoup(cineRequest.content, 'html.parser')
  editorial_sopa = BeautifulSoup(editorialesRequest.content, 'html.parser')

  descargarDiarios = diarios_sopa.find('a' , class_='btn btn-green btn-block')
  descargarCine = cine_sopa.find('a', class_='btn btn-green btn-block')
  descargarEditorial = editorial_sopa.find('a', class_='btn btn-green btn-block')

  diarioFrame = requests.get(descargarDiarios.get('href'))
  cineFrame = requests.get(descargarCine.get('href'))
  editorialFrame = requests.get(descargarEditorial.get('href'))
  

  cine = open("cine.csv", 'wb')
  cine.write(cineFrame.content)
  cine.close()

  diarios = open("diarios.csv", 'wb')
  diarios.write(diarioFrame.content)
  diarios.close()

  editorial = open("editorial.csv", 'wb')
  editorial.write(editorialFrame.content)
  editorial.close()



In [2]:
  cineCSV = pd.read_csv("cine.csv")
  diariosCSV = pd.read_csv("diarios.csv")
  editorialCSV = pd.read_csv("editorial.csv")


  cineCSV.rename(columns= {"cod_localidad": "Cod_Loc",
                           "id_provincia": "IdProvincia",
                           "id_departamento" : "IdDepartamento",
                           "categoria" : "Categoría",
                           "provincia" : "Provincia",
                           "localidad" : "Localidad",
                           "nombre" : "Nombre",
                           "direccion" : "Domicilio",
                           "cp" : "CP",
                           }, inplace = True)


In [3]:

datos = [diariosCSV,editorialCSV,cineCSV]

df5 = pd.concat(datos)

agrupado = open("agrupado.csv", 'wb')
agrupado.write(df5.to_csv(index=False).encode())
agrupado.close()

In [4]:
punto2 = {"Categoria" : [], "Diarios Impresos" : [], "Editoriales de Libros": [], "Salas de cine": []}

'''
cant__filtro = df5[""] == "Diarios Impresos"
cant_cines = df5[cant_cines_filtro]

cant_cines_filtro = df5["Categoría"] == "Editoriales de Libros"
cant_cines = df5[cant_cines_filtro]

cant_cines_filtro = df5["Categoría"] == "Diarios Impresos"
cant_cines = df5[cant_cines_filtro]
'''
provincias = df5['Provincia']
categorias = df5["Categoría"]

array_categorias = []
array_provincias = []

for provincia in provincias:
  if provincia not in array_provincias:
    array_provincias.append(provincia)

for categoria in categorias:
  if categoria not in array_categorias:
    array_categorias.append(categoria)


punto2["Categoria"].append("cantidad total")
punto2["Diarios Impresos"].append(len(df5[df5["Categoría"] == "Diarios Impresos"]))
punto2["Editoriales de Libros"].append(len(df5[df5["Categoría"] == "Editoriales de Libros"]))
punto2["Salas de cine"].append(len(df5[df5["Categoría"] == "Salas de cine"]))

for provincia in array_provincias:
  punto2["Categoria"].append( "cantidad en " + provincia)
  for categoria in array_categorias:
    condition = (df5["Categoría"] == categoria) & (df5["Provincia"] == provincia)
    cant = len(df5[condition])
    punto2[categoria].append (cant)


dataframe = pd.DataFrame(punto2)
print(dataframe)



                                      Categoria  Diarios Impresos  \
0                                cantidad total               245   
1                      cantidad en Buenos Aires                81   
2                         cantidad en Catamarca                 2   
3                             cantidad en Chaco                 5   
4                            cantidad en Chubut                 8   
5   cantidad en Ciudad Autónoma de Buenos Aires                21   
6                           cantidad en Córdoba                18   
7                        cantidad en Corrientes                 6   
8                        cantidad en Entre Ríos                16   
9                           cantidad en Formosa                 3   
10                            cantidad en Jujuy                 2   
11                         cantidad en La Pampa                 6   
12                         cantidad en La Rioja                 3   
13                          cantid

In [67]:
import numpy as np
punto3 = {"Provincia" : [], "Cant. de Pantallas" : [], "Cant. de butacas": [], "Cant. de espacios INCAA": []}


provincias = df5['Provincia']

array_provincias = []

for provincia in provincias:
  if provincia not in array_provincias:
    array_provincias.append(provincia)


punto3["Provincia"].append("cantidad total")
punto3["Cant. de Pantallas"].append(len(df5["pantallas"]))
punto3["Cant. de butacas"].append(len(df5["butacas"]))
punto3["Cant. de espacios INCAA"].append(len(df5["espacio_incaa"]))

array_cosas = ["Cant. de Pantallas", "Cant. de butacas", "Cant. de espacios INCAA",]
cat  = ["pantallas", "butacas", "espacio_incaa",]

for provincia in array_provincias:
  punto3["Provincia"].append(provincia)
  dfI = df5[df5["Provincia"] == provincia]
  dfI = dfI[dfI["Categoría"] == "Salas de cine"]
  #print(dfI)
  #print("----------------------------------------------------------------------")
  for categoria, index in enumerate(array_cosas):
    sum = 0
    if index == "Cant. de espacios INCAA" :
      for column_name, item in dfI[cat[categoria]].iteritems():
        if item == "Si" : sum = sum + 1
      punto3[index].append(sum)
    else:
      punto3[index].append(dfI[cat[categoria]].sum())


dataframe = pd.DataFrame(punto3)
print(dataframe)


                          Provincia  Cant. de Pantallas  Cant. de butacas  \
0                    cantidad total              1277.0            1277.0   
1                      Buenos Aires               362.0           92818.0   
2                         Catamarca                10.0            1657.0   
3                             Chaco                13.0            2302.0   
4                            Chubut                 9.0            2602.0   
5   Ciudad Autónoma de Buenos Aires               150.0           27940.0   
6                           Córdoba               110.0           22334.0   
7                        Corrientes                16.0            3170.0   
8                        Entre Ríos                14.0            3842.0   
9                           Formosa                 3.0             619.0   
10                            Jujuy                 5.0            2277.0   
11                         La Pampa                 5.0            1957.0   